# Determinantes Acústicos del Éxito Comercial en la Era del Streaming

### Duplicamos de nuestro repositorio de GitHub

In [1]:
!git clone https://github.com/Bmendez03/Regresion.git

fatal: destination path 'Regresion' already exists and is not an empty directory.


### Importamos las librerias necesarias

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm
from statsmodels.stats.outliers_influence import variance_inflation_factor
import tabulate

### Carga de Dataset

In [ ]:
print("--- 1. PREPARACIÓN DE DATOS ---")
try:
    # 1.1 Carga del dataset
    df = pd.read_csv(r'/content/Regresion/SpotifyFeatures.csv')
    print("Datos cargados correctamente.")
except:
    print("Error: Sube el archivo 'SpotifyFeatures.csv' a la carpeta de archivos.")
    exit()

### Selección de variables

In [4]:
# transormamos 'duration_ms' de milisegundos a minutos
df['duration_ms'] = df['duration_ms'] / 60000
# Selección de variables
vars_independientes_continuas = ['acousticness', 'danceability', 'energy', 'loudness', 'valence', 'duration_ms']
y = df['popularity'] # Variable Dependiente

# Crear el DataFrame de predictores (X)
X = df[vars_independientes_continuas]

NameError: name 'df' is not defined

### Diagnostico de multcolinealidad

In [ ]:
# La función VIF requiere que X tenga la constante (Intercepto), la quitaremos después.
X_vif = sm.add_constant(X)
vif_data = pd.DataFrame()
vif_data['feature'] = X_vif.columns
vif_data['VIF'] = [variance_inflation_factor(X_vif.values, i)
                   for i in range(X_vif.shape[1])]

# Mostrar VIF, excluyendo el 'const'
print("VIF por Variable (VIF > 5 indica posible problema):")
print(vif_data.loc[vif_data['feature'] != 'const'].sort_values(by='VIF', ascending=False).head(10).to_markdown(index=False))

### Estimación y resultados del modelo

In [ ]:
print("\n RESULTADOS DEL MODELO OLS ROBUSTO ")

# Añadir constante (Beta 0 / Intercepto) para el entrenamiento
X = sm.add_constant(X)

# Crear y entrenar el modelo
modelo_robusto = sm.OLS(y, X).fit()

# Imprimir el resumen estadístico completo
print(modelo_robusto.summary())

# Extracción de métricas clave solicitadas
print("\n--- MÉTRICAS DE CONFIABILIDAD ---")
print(f"R-cuadrado (Bondad de ajuste): {modelo_robusto.rsquared:.4f}")
print(f"R-cuadrado Ajustado: {modelo_robusto.rsquared_adj:.4f}")
print(f"AIC (Criterio de Akaike): {modelo_robusto.aic:.4f}")
print(f"Prob (F-statistic): {modelo_robusto.f_pvalue:.4e}")

### Prueba de hipótesis y dignóstico gráfico

In [ ]:
print("\n--- 4. ANÁLISIS DE SIGNIFICANCIA (P-value) ---")
# Solo mostramos las variables continuas y la primera dummy de cada categórica
for variable in ['acousticness', 'danceability', 'energy', 'loudness', 'valence', 'tempo','duration_ms', 'genre_Alternative', 'key_A#', 'mode_Minor', 'time_signature_1/4']:
    try:
        pvalue = modelo_robusto.pvalues[variable]
        significativo = "SIGNIFICATIVO" if pvalue < 0.05 else "NO SIGNIFICATIVO"
        print(f"Variable: {variable:20} | P-value: {pvalue:.4f} -> {significativo}")
    except KeyError:
        # Esto sucede si la categoría 'genre_Alternative' no existe (por ejemplo, si se eliminó)
        pass

In [ ]:
# Gráfica de distribución de residuales
plt.figure(figsize=(12, 5))
plt.subplot(1, 2, 1)
sns.histplot(modelo_robusto.resid, bins=50, kde=True, color='red')
plt.title('Distribución de los Residuales')
plt.xlabel('Error')

In [ ]:
# Gráfica de Homocedasticidad (Residuales vs. Valores Predichos)
plt.subplot(1, 2, 2)
plt.scatter(modelo_robusto.fittedvalues, modelo_robusto.resid, alpha=0.5)
plt.axhline(0, color='red', linestyle='--')
plt.title('Residuales vs. Valores Predichos (Homocedasticidad)')
plt.xlabel('Valores Predichos')
plt.ylabel('Residuales')
plt.tight_layout()
plt.show()